<a href="https://colab.research.google.com/github/Joonsung-Byun/ktcloud_genai/blob/main/20251015_%EB%AF%B8%EC%85%98_%EC%9E%84%EB%B2%A0%EB%94%A9_%EB%AA%A8%EB%8D%B8_%EB%A7%8C%EB%93%A4%EA%B8%B0_%EB%B3%80%EC%A4%80%EC%84%B1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **임베딩 모델 미세 튜닝하기**



---


- 💡 **NOTE**
    - 이 노트북의 코드를 실행하려면 GPU를 사용하는 것이 좋습니다. 구글 코랩에서는 **런타임 > 런타임 유형 변경 > 하드웨어 가속기 > T4 GPU**를 선택하세요.


---



In [1]:
# 깃허브에서 위젯 상태 오류를 피하기 위해 진행 표시줄을 나타내지 않도록 설정합니다.
import os
import tqdm
from transformers.utils import logging

# tqdm 비활성화
os.environ["DISABLE_TQDM"] = "1"

logging.disable_progress_bar()

In [ ]:
%%capture
# datasets를 MTEB 호환 버전 설치
!pip install 'datasets>=2.19.0,<3.0.0'

# pydantic을 MTEB 호환 버전 설치
!pip install 'pydantic>=2.0,<2.6'

# MTEB 최신 버전 설치
!pip install 'mteb>=1.39.0'



---



## **미세 튜닝(Fine-tuning) 이해**



### **미세 튜닝(Fine-tuning)이란**?

- **정의** : 이미 방대한 데이터로 학습된 **사전 학습 모델**(Pre-trained Model)을 가져와, 필요한 특정 작업(Task)에 맞게 소량의 데이터로 추가 학습시켜 성능을 최적화하는 과정

### **왜 미세 튜닝을 할까?**

- **시간과 비용 절약**:
    - 이미 만들어진 모델을 활용하므로 부담을 획기적으로 줄여줄 수 있다.
- **높은 성능 달성**:
    - 사전 학습 모델은 이미 언어의 문법, 문맥, 의미 등 방대한 지식을 갖추고 있어 이 지식을 바탕으로 특정 작업에 맞게 약간만 조정해도 매우 높은 성능을 낼 수 있다.
- **데이터 부족 문제 해결**:
    - 특정 작업을 위한 대규모 데이터셋을 구축하기는 어렵다. 미세 튜닝은 상대적으로 적은 양의 데이터만으로도 모델을 효과적으로 학습시킬 수 있게 해준다.

### **미세 튜닝 과정**

1. **사전 학습 모델 선택**:
    - 해결하려는 작업과 가장 **관련성이 높은 범용 모델**(예: BERT, GPT, KoELECTRA 등)을 선택

2. **데이터셋 준비**:
    - 풀고 싶은 특정 작업에 맞는 **소규모의 레이블링된 데이터셋을 준비** (예: 영화 리뷰 감성 분류 데이터, 법률 문서 개체명 인식 데이터)

3. **모델 구조 변경**:
    - 사전 학습 모델의 맨 마지막 단에 풀고 싶은 작업에 맞는 **새로운 층(Layer)을 추가**. 예를 들어, 문장 분류를 하고 싶다면 분류를 위한 '분류 헤드(Classification Head)'를 추가한다.

4. **추가 학습**:
    - 준비된 데이터셋으로 모델을 추가 학습시킨다. 이때, 기존의 지식이 손상되지 않도록 **낮은 학습률**(learning rate)을 사용하는 것이 매우 중요하다. 이는 기존 지식은 유지하면서 새로운 작업에 맞게 가중치를 '미세하게' 조정하는 효과를 준다.

5. **평가**:
    - 학습된 모델이 해당 작업에서 얼마나 좋은 성능을 보이는지 평가한다.

### **사전 학습 vs. 미세 튜닝**

|구분	|사전 학습 (Pre-training)	|미세 튜닝(Fine-tuning)|
|---|---|---|
|**목표**|	언어 자체에 대한 범용적인 이해 능력 확보|	특정 작업(분류, 요약 등)에 대한 전문성 확보|
|**데이터**|	대규모의 레이블 없는 텍스트 (e.g., 웹 전체)|	소규모의 레이블 있는 특정 도메인 텍스트|
|**학습 규모**|	수 주 ~ 수 개월, 수백 대의 GPU 필요|	수 분 ~ 수 시간, 소수의 GPU로 가능|
|**결과물**|	범용 언어 모델 (Foundation Model)|	특정 작업에 특화된 모델 (Specialized Model)|

### **미세 튜닝의 어려운점**

|구분|어려움|영향|해결 난이도|
|---|---|---|---|
|1. 데이터 품질|양질의 레이블링 데이터 확보 어려움|모델 성능 저하|⭐⭐⭐⭐⭐|
|2. 과적합(Overfitting)|작은 데이터셋에서 과적합 발생|일반화 성능 저하|⭐⭐⭐⭐|
|3. 계산 자원|GPU 메모리, 학습 시간, 비용 문제|실험 제한|⭐⭐⭐⭐|
|4. 하이퍼파라미터 튜닝|최적의 learning rate, epoch 찾기 어려움|성능 편차 큼|⭐⭐⭐⭐|
|5. 재앙적 망각|사전학습 지식 손실|다른 태스크 성능 저하|⭐⭐⭐|
|6. 클래스 불균형|데이터 분포 편향|소수 클래스 예측 실패|⭐⭐⭐|


1. **데이터 품질 문제**
    - 전문가 레이블링 비용이 매우 높음
    - 레이블 오류가 모델에 직접 전파
    - 도메인 특화 데이터 수집의 어려움

2. **과적합 위험**
    - 사전학습 모델은 파라미터가 매우 많음 (BERT: 110M개)
    - 작은 학습 데이터로는 특정 패턴만 암기
    - validation loss와 training loss의 격차 발생

3. **계산 자원 제약**
    - 대형 모델은 최소 16GB GPU 필요
    - 배치 크기 제한으로 학습 불안정
    - 실험 반복 시간과 비용 증가

4. **하이퍼파라미터 민감성**
    - Learning rate가 너무 높으면 → 재앙적 망각
    - Learning rate가 너무 낮으면 → 학습 안 됨
    - 최적값이 태스크마다 다름

5. **재앙적 망각** (Catastrophic Forgetting)
    - 새 태스크 학습 시 이전 지식 손실
    - 사전학습된 언어 이해 능력 저하
    - 다중 태스크 동시 학습의 어려움

6. **클래스 불균형**
    - 실제 데이터는 불균형한 경우가 많음
    - 다수 클래스에 편향된 예측
    - 소수 클래스의 중요도가 높은 경우 문제



---



## **임베딩 모델 미세 튜닝하기**

In [ ]:
# 1) 데이터 준비
from datasets import load_dataset

# GLUE에서 MNLI 데이터셋을 로드합니다.
# 0 = 수반, 1 = 중립, 2 = 모순
train_dataset = load_dataset("glue", "mnli", split="train").select(range(50_000))
train_dataset = train_dataset.remove_columns("idx")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

mnli/train-00000-of-00001.parquet:   0%|          | 0.00/52.2M [00:00<?, ?B/s]

mnli/validation_matched-00000-of-00001.p(…):   0%|          | 0.00/1.21M [00:00<?, ?B/s]

mnli/validation_mismatched-00000-of-0000(…):   0%|          | 0.00/1.25M [00:00<?, ?B/s]

...

Generating train split:   0%|          | 0/392702 [00:00<?, ? examples/s]

Generating validation_matched split:   0%|          | 0/9815 [00:00<?, ? examples/s]

Generating validation_mismatched split:   0%|          | 0/9832 [00:00<?, ? examples/s]

Generating test_matched split:   0%|          | 0/9796 [00:00<?, ? examples/s]

Generating test_mismatched split:   0%|          | 0/9847 [00:00<?, ? examples/s]

In [ ]:
train_dataset[2]
# 전제(premise)와 가설(hypothesis)이 서로 관련되고 의미가 거의 동일: 0=수반에 해당

{'premise': 'One of our number will carry out your instructions minutely.',
 'hypothesis': 'A member of my team will execute your orders with immense precision.',
 'label': 0}

In [ ]:
import pandas as pd

df = pd.DataFrame(
    {
        "sentence1": train_dataset["premise"],
        "sentence2": train_dataset["hypothesis"],
        "label": train_dataset["label"]
    }
)
df.head()

,sentence1,sentence2,label
0,Conceptually cream skimming has two basic dime...,Product and geography are what make cream skim...,1
1,you know during the season and i guess at at y...,You lose the things to the following level if ...,0
2,One of our number will carry out your instruct...,A member of my team will execute your orders w...,0
3,How do you know? All this is their information...,This information belongs to them.,0
4,yeah i tell you what though if you go price so...,The tennis shoes have a range of prices.,1


In [ ]:
# df['label']별 빈도 수
df['label'].value_counts()

,count
label,
2,17884
0,16875
1,15241


### **예제: 지도 학습 방법**

- **SFT: Supervised Fine-Tuning**
    - **SFT는 사전 학습된 모델을 레이블이 있는 데이터셋으로 추가 학습시켜 특정 작업에 맞게 조정하는 과정**
    - **"Supervised"의 의미**: 레이블된 데이터를 사용하여 학습 과정을 안내한다는 의미. 모델은 명시적인 피드백을 통해 **특정 입력에 대한 원하는 출력으로 매핑하는 방법을 학습**한다
    - 임베딩 모델 미세 튜닝에서는 SFT 대신 그냥 "Fine-tuning" 용어를 사용한다.(업계 관습?)

- **임베딩 모델을 미세 튜닝하는 가장 간단한 방법**
    - 이전에 했던 모델 훈련 과정을 그대로 따라하는 것
    - **모델 변경**: **bert-base-uncased** --> **sentence-transformers/all-MiniLM-L6-v2**(작고 속도가 좋다)


In [ ]:
# 1) 데이터 로드
from datasets import load_dataset
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator

# GLUE에서 MNLI 데이터셋을 로드합니다.
# 0 = 수반, 1 = 중립, 2 = 모순
train_dataset = load_dataset("glue", "mnli", split="train").select(range(50_000))
train_dataset = train_dataset.remove_columns("idx")

# STSB를 위해 임베딩 유사도 평가자를 만듭니다.
val_sts = load_dataset('glue', 'stsb', split='validation')
evaluator = EmbeddingSimilarityEvaluator(
    sentences1=val_sts["sentence1"],
    sentences2=val_sts["sentence2"],
    scores=[score/5 for score in val_sts["label"]],
    main_similarity="cosine",
    similarity_fn_names=["cosine", "euclidean", "manhattan", "dot"]
)

stsb/train-00000-of-00001.parquet:   0%|          | 0.00/502k [00:00<?, ?B/s]

stsb/validation-00000-of-00001.parquet:   0%|          | 0.00/151k [00:00<?, ?B/s]

stsb/test-00000-of-00001.parquet:   0%|          | 0.00/114k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/5749 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1500 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1379 [00:00<?, ? examples/s]

In [ ]:
# 2) 모델 훈련 : 모델 정의 + 손실 함수 + 훈련 매개변수 + 모델 훈련
from sentence_transformers import losses, SentenceTransformer
from sentence_transformers.trainer import SentenceTransformerTrainer
from sentence_transformers.training_args import SentenceTransformerTrainingArguments

# 모델
embedding_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# 손실 함수
train_loss = losses.MultipleNegativesRankingLoss(model=embedding_model)

# 훈련 매개변수
args = SentenceTransformerTrainingArguments(
    output_dir="finetuned_embedding_model",
    num_train_epochs=1,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    warmup_steps=100,
    fp16=True,
    eval_steps=100,
    logging_steps=100,
    report_to=[]
)

# 모델 훈련
trainer = SentenceTransformerTrainer(
    model=embedding_model,
    args=args,
    train_dataset=train_dataset,
    loss=train_loss,
    evaluator=evaluator
)
trainer.train()

Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

dataset = dataset.select_columns(['hypothesis', 'entailment', 'contradiction'])


Step,Training Loss
100,0.158300
200,0.113100
300,0.122400
400,0.119600
500,0.110400
600,0.101700
700,0.121300
800,0.101700
900,0.102500
1000,0.104300


TrainOutput(global_step=1563, training_loss=0.11035708319431532, metrics={'train_runtime': 123.3509, 'train_samples_per_second': 405.348, 'train_steps_per_second': 12.671, 'total_flos': 0.0, 'train_loss': 0.11035708319431532, 'epoch': 1.0})

In [ ]:
# 훈련된 모델을 평가합니다.
evaluator(embedding_model)

{'pearson_cosine': 0.8492865095648342,
 'spearman_cosine': 0.8491099329549162,
 'pearson_euclidean': 0.852715729454054,
 'spearman_euclidean': 0.8491099329549162,
 'pearson_manhattan': 0.8516469978649773,
 'spearman_manhattan': 0.8479777026361854,
 'pearson_dot': 0.8492865060864098,
 'spearman_dot': 0.8491099329549162,
 'pearson_max': 0.852715729454054,
 'spearman_max': 0.8491099329549162}

In [ ]:
# 사전 훈련된 모델을 평가합니다.
original_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
evaluator(original_model)

{'pearson_cosine': 0.8696194527908605,
 'spearman_cosine': 0.8671631197908374,
 'pearson_euclidean': 0.8678715930680355,
 'spearman_euclidean': 0.8671631197908374,
 'pearson_manhattan': 0.8670398986664181,
 'spearman_manhattan': 0.8663946139224048,
 'pearson_dot': 0.869619451987714,
 'spearman_dot': 0.8671631197908374,
 'pearson_max': 0.8696194527908605,
 'spearman_max': 0.8671631197908374}

⚠️ **VRAM 비우기**

In [ ]:
import gc
import torch

gc.collect()
torch.cuda.empty_cache()



---



### **예제: 증식 SBERT**

- **임베딩 모델을 훈련하거나 미세 튜닝하는데 어려운 점**
    - 많은 양의 훈련 데이터가 필요하다는 점
    - 많은 모델이 10억 개 이상의 문장 쌍에서 훈련됨
    
- **증식(Argumented) SBERT** : https://arxiv.org/abs/2010.08240
    - **레이블이 있는 데이터가 적을 때, 문장 임베딩 모델을 미세 튜닝할 수 있도록 데이터를 증식하는 방법**
    - BERT vs. SBERT
        - **BERT** : 두 문장을 함께 입력받아 관계 예측 (**Cross-Encoder**)
        - **SBERT** : 각 문장을 독립적으로 임베딩하여 유사도 계산 (**Bi-Encoder**)
    - 참고: https://wikidocs.net/156176
- **증식(Argumented) SBERT의 과정(4단계 파이프라인)**
    1. **Gold 데이터셋으로 크로스 인코더 학습**
        - 정답이 달린 소량의 데이터(Gold 데이터셋)를 사용해서, 두 문장 관계(같은 의미/다름 등)를 잘 판단하는 **크로스 인코더**(BERT)를 미세조정
        - 비유: 국어 선생님에게 ‘참/거짓 문제’를 1만 개 가르쳐서, 문장 의미 판단을 잘하도록 훈련하는 단계
    2. **새로운 문장쌍 생성**
        - Gold 외의 문장을 이용해 다양한 문장쌍을 무작위로 조합
        - 비유: 선생님이 배운 내용 바탕으로 비슷한 문제를 스스로 만들어보기
    3. **크로스 인코더로 레이블 부여 (Silver 데이터셋**, 레이블은 있지만 정답이 아닐 수 있다)
        - 새 문장쌍마다 “이건 의미가 비슷/다름”을 크로스 인코더가 예측해서 점수 부여 → silver dataset 완성
        - 비유: 선생님이 새 문제에 답을 달아줌 (하지만 완벽히 정답은 아닐 수도 있음)
    4. **Gold + Silver로 바이 인코더(SBERT) 학습**
        - 이렇게 확장된 대량 데이터로 SBERT(문장 임베딩용 모델) 학습 → 빠르고 효율적
        - 비유: 학생이 ‘선생님 답안’을 보고 대량 연습을 통해 유사도 감각을 체득


![출처: 핸즈온 LLM](https://drive.google.com/uc?export=view&id=15C_AsJyz7YE2phpatFpnhuso8YJAyxGn)

- ⚠️ wandb 완전 비활성화
    - wandb : 모델 훈련 과정을 클라우드 대시보드에서 실시간으로 모니터링하게 해주는 서비스
    - Colab에는 보통 wandb가 미리 설치되어 있고, sentence-transformers(또는 내부에서 쓰는 transformers/Trainer)가 wandb가 설치돼 있으면 자동으로 로깅을 켜려 시도함
    - 비활성화 시킴

In [ ]:
# ✅ 이 셀을 "항상" 최상단(모델/라이브러리 import 전에) 실행하세요.
import os
os.environ["WANDB_DISABLED"] = "true"   # wandb 완전 비활성화
os.environ["WANDB_MODE"] = "disabled"   # (중복 안전장치)
os.environ["WANDB_SILENT"] = "true"    # 로그도 조용히

import warnings
warnings.filterwarnings("ignore", category=SyntaxWarning)

**단계 1:** 크로스 인코더를 미세 튜닝한다.(골드 데이터셋 만듦)
- 제한적인 환경을 흉내내기 위해 50,000 --> 10,000개의 문서만 사용한다.

In [ ]:
import pandas as pd
from tqdm import tqdm
from datasets import load_dataset, Dataset
from sentence_transformers import InputExample
from sentence_transformers.datasets import NoDuplicatesDataLoader

# 크로스 인코더를 위해 10,000개의 문서로 구성된 데이터셋을 만듭니다.
dataset = load_dataset("glue", "mnli", split="train").select(range(10_000))
# (중립/모순)=0, (수반)=1
mapping = {2: 0, 1: 0, 0:1}

# 데이터 로더
gold_examples = [
    InputExample(texts=[row["premise"], row["hypothesis"]], label=mapping[row["label"]])
    for row in tqdm(dataset)
]
gold_dataloader = NoDuplicatesDataLoader(gold_examples, batch_size=32)

# 데이터 처리를 쉽게 하기 위해 판다스 데이터프레임을 만듭니다.
gold = pd.DataFrame(
    {
    'sentence1': dataset['premise'],
    'sentence2': dataset['hypothesis'],
    'label': [mapping[label] for label in dataset['label']]
    }
)

In [ ]:
from sentence_transformers.cross_encoder import CrossEncoder

# 골드 데이터셋에서 크로스 인코더를 훈련합니다.
cross_encoder = CrossEncoder('bert-base-uncased', num_labels=2)
cross_encoder.fit(
    train_dataloader=gold_dataloader,
    epochs=1,
    show_progress_bar=True,
    warmup_steps=100,
    use_amp=False
)

**단계 2:** 새로운 문장 쌍을 만듭니다.(실버 데이터셋 만듦)

In [ ]:
# 크로스 인코더로 레이블을 예측할 실버 데이터셋을 만듭니다.
silver = load_dataset("glue", "mnli", split="train").select(range(10_000, 50_000))
pairs = list(zip(silver['premise'], silver['hypothesis']))

**단계 3:** 미세 튜닝된 크로스 인코더로 새로운 문장 쌍(실버 데이터셋)에 레이블을 할당합니다.

In [ ]:
import numpy as np

# 미세 튜닝된 크로스 인코더를 사용해 문장 쌍에 레이블을 할당합니다.
output = cross_encoder.predict(pairs, apply_softmax=True,
                               show_progress_bar=True)
silver = pd.DataFrame(
    {
        "sentence1": silver["premise"],
        "sentence2": silver["hypothesis"],
        "label": np.argmax(output, axis=1)
    }
)
silver.head(2)

**단계 4:** 확장된 데이터셋(골드 데이터셋 + 실버 데이터셋)으로 바이 인코더(SBERT)를 훈련합니다.

In [ ]:
# 골드 데이터셋과 실버 데이터셋을 합칩니다.
data = pd.concat([gold, silver], ignore_index=True, axis=0)
data = data.drop_duplicates(subset=['sentence1', 'sentence2'], keep="first")
train_dataset = Dataset.from_pandas(data, preserve_index=False)
print(len(train_dataset))

In [ ]:
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator

# STSB를 위한 임베딩 유사도 평가자를 만듭니다.
val_sts = load_dataset('glue', 'stsb', split='validation')
evaluator = EmbeddingSimilarityEvaluator(
    sentences1=val_sts["sentence1"],
    sentences2=val_sts["sentence2"],
    scores=[score/5 for score in val_sts["label"]],
    main_similarity="cosine",
    similarity_fn_names=["cosine", "euclidean", "manhattan", "dot"]
)

In [ ]:
from sentence_transformers import losses, SentenceTransformer
from sentence_transformers.trainer import SentenceTransformerTrainer
from sentence_transformers.training_args import SentenceTransformerTrainingArguments

# 모델
embedding_model = SentenceTransformer('bert-base-uncased')

# 손실 함수
train_loss = losses.CosineSimilarityLoss(model=embedding_model)

# 훈련 매개변수
args = SentenceTransformerTrainingArguments(
    output_dir="augmented_embedding_model",
    num_train_epochs=1,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    warmup_steps=100,
    fp16=True,
    eval_steps=100,
    logging_steps=100,
    report_to=[]
)

# 모델 훈련
trainer = SentenceTransformerTrainer(
    model=embedding_model,
    args=args,
    train_dataset=train_dataset,
    loss=train_loss,
    evaluator=evaluator
)
trainer.train()

In [ ]:
# 훈련된 모델을 평가합니다.
evaluator(embedding_model)

# 결론: 데이터의 20%만 사용하고 0.71~점수를 얻음

In [ ]:
trainer.accelerator.clear()

**단계 5**: 실버 데이터셋을 사용하지 않고 평가합니다.

In [ ]:
# 골드 데이터셋만 사용합니다.
data = pd.concat([gold], ignore_index=True, axis=0)
data = data.drop_duplicates(subset=['sentence1', 'sentence2'], keep="first")
train_dataset = Dataset.from_pandas(data, preserve_index=False)

# 모델
embedding_model = SentenceTransformer('bert-base-uncased')

# 손실 함수
train_loss = losses.CosineSimilarityLoss(model=embedding_model)

# 훈련 매개변수
args = SentenceTransformerTrainingArguments(
    output_dir="gold_only_embedding_model",
    num_train_epochs=1,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    warmup_steps=100,
    fp16=True,
    eval_steps=100,
    logging_steps=100,
    report_to=[]
)

# 모델 훈련
trainer = SentenceTransformerTrainer(
    model=embedding_model,
    args=args,
    train_dataset=train_dataset,
    loss=train_loss,
    evaluator=evaluator
)
trainer.train()

In [ ]:
# 훈련된 모델을 평가합니다.
evaluator(embedding_model)

# 결론: 골드 데이터셋만 사용하는 것보다는 실버 데이터셋을 함꼐 사용하는 것이 모델 성능에 좋다.

실버 데이터셋과 골드 데이터셋을 모두 사용했을 때와 비교하면 골드 데이터셋만 사용한 경우 모델의 성능이 감소합니다!

⚠️ **VRAM 비우기**

In [ ]:
import gc
import torch

gc.collect()
torch.cuda.empty_cache()



---



## **비지도 학습**

- **비지도 학습**(Unsupervised learning)
    - 사전에 정의된 레이블 없이 모델을 훈련하는 기법
    - 레이블이 없는 대량의 텍스트만으로 고품질의 문장 임베딩을 생성하는 것을 목표로함

- **대표적인 비지도 학습 방법**
    - **SimCSE**(Simple Contrastive Learning of Sentence Embeddings): https://arxiv.org/abs/2104.08821
        - 드롭아웃(dropout)을 최소한의 데이터 증강 기법으로 활용하여, 동일한 문장을 두 번 인코딩한 결과를 긍정 쌍(positive pair)으로 삼아 대조 학습(contrastive learning)을 수행하는 방식

    - **CT**(Contrastive Tension):
        - 한 문장을 다른 모델(e.g., 번역 모델)을 통해 의미가 동일한 새로운 문장으로 생성하여 긍정 쌍을 만들고, 이 긍정 쌍의 유사도는 높이는 동시에 배치 내 모든 다른 문장들과의 유사도는 낮추는 강력한 대조 학습 방식

    - **TSDAE**(Transformer-based Sequential Denoising Auto-Encoder): https://arxiv.org/abs/2104.06979
        - 문장의 일부 단어를 **삭제하거나 순서를 섞는 등 손상(corrupt)시킨 뒤**, 원본 문장을 복원하도록 학습하는 **노이즈 제거 오토인코더(Denoising AutoEncoder) 구조**를 사용함
        - 인코더는 손상된 문장으로부터 의미 있는 임베딩을 생성해야 하고, 디코더는 이 임베딩 정보만을 이용해 완벽한 원본 문장을 재구성해야 함.
        - 이 과정에서 인코더는 문장의 핵심 의미를 압축적으로 담아내는 능력을 학습하게 되며, 이렇게 학습된 인코더가 고품질의 문장 임베딩을 생성함

    - **GPL**(Generative Pseudo-Labeling): https://arxiv.org/abs/2112.07577
        - 비지도 데이터를 활용해 **대규모의 고품질 학습 데이터를 자동으로 생성**(Generative Pseudo-Labeling)하여 모델을 학습시키는 파이프라인
    - 등 다양한 방법 존재

<img src="https://drive.google.com/uc?export=view&id=1dDuPhEKv8O2o85TgSN_6Z5WsUfWYc34h" width="80%">

### **예제: TSDAE**
- TSDAE(Transformer-based Sequential Denoising Auto-Encoder)
- **레이블 데이터가 전혀 없다고 가정**하며 인공적으로 레이블을 만들 필요가 없다.
- **TSDAE는 입력 문장에 잡음을 추가해 단어를 일정 비율 제거한다는 아이디어에 기반함**

- **핵심 개념**: 문장 임베딩이 정확할수록 재구성 문장이 정확하다!
    - 잡음이 추가된 오염된 문장을 인코더와 풀링 층에 통과시켜서 문장 임베딩에 매핑함
    - 이 문장 임베딩을 사용해 디코더가 오염된 문장에서 인공적인 잡음이 없는 원본 문장을 재구성함
    - 마스킹 된특정 단어를 재구성하는 마스크드 언어 모델링과 매우 비슷하지만, 마스킹 된특정 단어를 재구성이 아니라 **전체 문장을 재구성한다.**


- **잡음제거 과정에 사용할 토크나이저 다운로드한다.**

In [ ]:
# 추가적인 토크나이저를 다운로드합니다.
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')

- **데이터에서 두 문장(전제,가설)을 합치고 비지도 학습을 흉내내기 위해 레이브을 제거한다.**

In [ ]:
from tqdm import tqdm
from datasets import Dataset, load_dataset
from sentence_transformers.datasets import DenoisingAutoEncoderDataset

# 전제와 가설을 하나의 문장으로 연결합니다. (레이블 사용 안함)
mnli = load_dataset("glue", "mnli", split="train").select(range(25_000))
flat_sentences = list(mnli["premise"]) + list(mnli["hypothesis"])
print(len(flat_sentences))  # ?건

# 입력 데이터에 잡음을 추가합니다.
damaged_data = DenoisingAutoEncoderDataset(list(set(flat_sentences)))
print(len(damaged_data))    # ?건

# 데이터셋을 만듭니다.
train_dataset = {"damaged_sentence": [], "original_sentence": []}
for data in tqdm(damaged_data):
    train_dataset["damaged_sentence"].append(data.texts[0])
    train_dataset["original_sentence"].append(data.texts[1])
train_dataset = Dataset.from_dict(train_dataset)

In [ ]:
train_dataset[0]

In [ ]:
df = pd.DataFrame(
    {
    'damaged_sentence': train_dataset['damaged_sentence'],
    'original_sentence': train_dataset['original_sentence']
    }
)
df.head()

- **평가자 만들기**(evaluator)

In [ ]:
# 평가자 만들기
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator

# STSB를 위한 임베딩 유사도 평가자를 만듭니다.
val_sts = load_dataset('glue', 'stsb', split='validation')
evaluator = EmbeddingSimilarityEvaluator(
    sentences1=val_sts["sentence1"],
    sentences2=val_sts["sentence2"],
    scores=[score/5 for score in val_sts["label"]],
    main_similarity="cosine",
    similarity_fn_names=["cosine", "euclidean", "manhattan", "dot"]
)

- **모델 지정**
    - 풀링 레이어에 ('cls') 토큰 추가: 훈련시 위치 정보를 잃지 않게 하기 위해
    - cls :classification토큰

In [ ]:
# 모델 지정 : 풀링 레이어에 ('cls') 토큰 추가: 훈련시 위치 정보를 잃지 않게 하기 위해
from sentence_transformers import models, SentenceTransformer

# 임베딩 모델을 만듭니다.
word_embedding_model = models.Transformer('bert-base-uncased')
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(), 'cls')
embedding_model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

# modules = [word_embedding_model, pooling_model]
#                ↓                      ↓
#         1단계: 토큰 임베딩      2단계: 문장 임베딩
#        각 토큰이 768 차원벡터       1개의 768차원 벡터

- **손실 함수 지정**
    - 원본 문장을 재구성하는 손실 함수(DenoisingAutoEncoderLoss) 사용
    - tie_encoder_decoder=True : 인코더의 임베딩 층과 디코더의 출력 층의 가중치를 별도로 훈련하지 않고 동일한 가중치를 공유함 --> 한 층의 가중치를 업데이트하면 다른 층에도 반영됨

In [ ]:
from sentence_transformers import losses

# 잡음제거 오토 인코더 손실
train_loss = losses.DenoisingAutoEncoderLoss(
    embedding_model, tie_encoder_decoder=True
)
train_loss.decoder = train_loss.decoder.to("cuda")

- **모델 훈련**

In [ ]:
from sentence_transformers.trainer import SentenceTransformerTrainer
from sentence_transformers.training_args import SentenceTransformerTrainingArguments

# 훈련 매개변수
args = SentenceTransformerTrainingArguments(
    output_dir="tsdae_embedding_model",
    num_train_epochs=1,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=100,
    fp16=True,
    eval_steps=100,
    logging_steps=100,
    report_to=[]
)

# 모델 훈련
trainer = SentenceTransformerTrainer(
    model=embedding_model,
    args=args,
    train_dataset=train_dataset,
    loss=train_loss,
    evaluator=evaluator
)
trainer.train()

In [ ]:
# 훈련된 모델을 평가합니다.
evaluator(embedding_model)